<a href="https://colab.research.google.com/github/TushikaTibrewal/Disaster-Tweet/blob/main/Disaster_Tweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import string
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import nltk

**CLEANING**

In [2]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
ds=pd.read_csv('train.csv')
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    # Remove mentions (@user)
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word.lower() not in stop_words]
    text = ' '.join(tokens)
    return text

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
ds['text'] = ds['text'].astype(str).apply(clean_text)

# Preview cleaned data
ds.head()

In [ ]:
ds = ds.fillna('')

**EXPLORATORY DATA ANALYSIS**

In [ ]:
from collections import Counter
from textblob import TextBlob
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:

# Plot class distribution
plt.figure(figsize=(6,4))
sns.countplot(x='target', data=ds)
plt.title('Disaster vs Non-Disaster Tweet Distribution')
plt.xlabel('Target (0: Non-Disaster, 1: Disaster)')
plt.ylabel('Number of Tweets')
plt.show()

# Generate word clouds for each class
disaster_text = ' '.join(ds[ds['target'] == 1]['text'].astype(str).tolist())
nondisaster_text = ' '.join(ds[ds['target'] == 0]['text'].astype(str).tolist())

disaster_wc = WordCloud(width=800, height=400, background_color='white').generate(disaster_text)
nondisaster_wc = WordCloud(width=800, height=400, background_color='white').generate(nondisaster_text)

plt.figure(figsize=(12, 6))
plt.subplot(1,2,1)
plt.imshow(disaster_wc, interpolation='bilinear')
plt.axis('off')
plt.title('Disaster Tweets Word Cloud')
plt.subplot(1,2,2)
plt.imshow(nondisaster_wc, interpolation='bilinear')
plt.axis('off')
plt.title('Non-Disaster Tweets Word Cloud')
plt.show()

# Text length exploration
ds['text_len'] = ds['text'].astype(str).apply(len)
plt.figure(figsize=(8,5))
sns.histplot(ds, x='text_len', hue='target', element='step', stat='density', common_norm=False, bins=30)
plt.title('Tweet Length Distribution by Class')
plt.xlabel('Text Length')
plt.ylabel('Density')
plt.show()

# Most frequent words (excluding stopwords)
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def get_most_common_words(text_series, n=20):
    words = ' '.join(text_series).lower().split()
    clean_words = [word for word in words if word not in stop_words and word.isalpha()]
    return Counter(clean_words).most_common(n)

print('Most Common words in Disaster Tweets:')
print(get_most_common_words(ds[ds['target']==1]['text']))
print('\nMost Common words in Non-Disaster Tweets:')
print(get_most_common_words(ds[ds['target']==0]['text']))

# Sentiment pattern analysis
# Compute polarity using TextBlob
ds['polarity'] = ds['text'].astype(str).apply(lambda x: TextBlob(x).sentiment.polarity)

plt.figure(figsize=(8,5))
sns.histplot(ds, x='polarity', hue='target', element='step', stat='density', common_norm=False, bins=30)
plt.title('Sentiment Polarity Distribution by Class')
plt.xlabel('Sentiment Polarity')
plt.ylabel('Density')
plt.show()


**FEATURE ENGINEERING**

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

texts = ds['text'].astype(str)  # Change 'text' if your column has another name

# TF-IDF: Unigram (1,1)
vectorizer_uni = TfidfVectorizer(ngram_range=(1,1), max_features=1000)
X_uni = vectorizer_uni.fit_transform(texts)
print("Unigram TF-IDF shape:", X_uni.shape)

# TF-IDF: Bigram (1,2)
vectorizer_bi = TfidfVectorizer(ngram_range=(1,2), max_features=1000)
X_bi = vectorizer_bi.fit_transform(texts)
print("Bigram TF-IDF shape:", X_bi.shape)

# TF-IDF: Trigram (1,3)
vectorizer_tri = TfidfVectorizer(ngram_range=(1,3), max_features=1000)
X_tri = vectorizer_tri.fit_transform(texts)
print("Trigram TF-IDF shape:", X_tri.shape)

# Show sample features for each
print("Sample Unigram Features:", vectorizer_uni.get_feature_names_out()[:10])
print("Sample Bigram Features:", vectorizer_bi.get_feature_names_out()[:10])
print("Sample Trigram Features:", vectorizer_tri.get_feature_names_out()[:10])


**DATA TRAINING & EVALUATION**

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Prepare features and labels
X = X_uni  # or X_bi or X_tri as per your choice from TF-IDF
y = ds['target']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_results = {}

# 1. Logistic Regression
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_test)
model_results['Logistic Regression'] = {
    'accuracy': accuracy_score(y_test, y_pred_logreg),
    'precision': precision_score(y_test, y_pred_logreg),
    'recall': recall_score(y_test, y_pred_logreg),
    'f1': f1_score(y_test, y_pred_logreg),
    'confusion_matrix': confusion_matrix(y_test, y_pred_logreg)
}

# 2. Multinomial Naive Bayes
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
model_results['Multinomial Naive Bayes'] = {
    'accuracy': accuracy_score(y_test, y_pred_nb),
    'precision': precision_score(y_test, y_pred_nb),
    'recall': recall_score(y_test, y_pred_nb),
    'f1': f1_score(y_test, y_pred_nb),
    'confusion_matrix': confusion_matrix(y_test, y_pred_nb)
}

# 3. Support Vector Machine
svm = LinearSVC()
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
model_results['Support Vector Machine'] = {
    'accuracy': accuracy_score(y_test, y_pred_svm),
    'precision': precision_score(y_test, y_pred_svm),
    'recall': recall_score(y_test, y_pred_svm),
    'f1': f1_score(y_test, y_pred_svm),
    'confusion_matrix': confusion_matrix(y_test, y_pred_svm)
}

# Show and compare results
for model, metrics in model_results.items():
    print(f"\n--- {model} ---")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")
    print("Confusion Matrix:")
    print(metrics['confusion_matrix'])
    print()

# Optionally, detailed classification reports
print("\nClassification Reports:")
print("Logistic Regression\n", classification_report(y_test, y_pred_logreg))
print("Multinomial Naive Bayes\n", classification_report(y_test, y_pred_nb))
print("Support Vector Machine\n", classification_report(y_test, y_pred_svm))
